In [14]:
#Нейро-нечеткие системы
#Лабораторная работа №2

#Задача: кластеризация данных

In [2]:
%matplotlib inline
import os
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import Select, SelectMultiple, interactive,  Layout, Box, Label
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn import cluster
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN, AffinityPropagation, SpectralClustering
from itertools import cycle

In [3]:
#поиск всех файлов в папке
files = []
for file in os.listdir("."):
    if file.endswith('.csv'):
        files.append(file) 
        
if len(files) != 0:

    fileselect = Select(
        options=files,
        value=files[0],
        description='Select file:',
        disabled=False
    )
fileselect

Select(description='Select file:', options=('Anatase UV.csv', 'avocado.csv', 'cancer_b.csv', 'cancer_m.csv', '…

In [4]:
df = pd.read_csv(fileselect.value, delimiter=",")
df.describe()

,Unnamed: 0,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,year
count,18249.000000,18249.000000,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,1.824900e+04,18249.000000,18249.000000
mean,24.232232,1.405978,8.506440e+05,2.930084e+05,2.951546e+05,2.283974e+04,2.396392e+05,1.821947e+05,5.433809e+04,3106.426507,2016.147899
std,15.481045,0.402677,3.453545e+06,1.264989e+06,1.204120e+06,1.074641e+05,9.862424e+05,7.461785e+05,2.439660e+05,17692.894652,0.939938
min,0.000000,0.440000,8.456000e+01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,2015.000000
25%,10.000000,1.100000,1.083858e+04,8.540700e+02,3.008780e+03,0.000000e+00,5.088640e+03,2.849420e+03,1.274700e+02,0.000000,2015.000000
50%,24.000000,1.370000,1.073768e+05,8.645300e+03,2.906102e+04,1.849900e+02,3.974383e+04,2.636282e+04,2.647710e+03,0.000000,2016.000000
75%,38.000000,1.660000,4.329623e+05,1.110202e+05,1.502069e+05,6.243420e+03,1.107834e+05,8.333767e+04,2.202925e+04,132.500000,2017.000000
max,52.000000,3.250000,6.250565e+07,2.274362e+07,2.047057e+07,2.546439e+06,1.937313e+07,1.338459e+07,5.719097e+06,551693.650000,2018.000000


In [5]:
ddf = df.select_dtypes(include=['number'])
form_item_layout = Layout(
    display='flex',
    flex_flow='row',
    justify_content='space-between'
)

form_items = [
    Box([Label(value='Первая колонка'), Select(
    options=list(ddf.columns),
    disabled=False
)], layout=form_item_layout),
 Box([Label(value='Вторая колонка'), Select(
    options=list(ddf.columns),
    disabled=False
)], layout=form_item_layout),
]

form = Box(form_items, layout=Layout(
    display='flex',
    flex_flow='column',
    border='solid 2px',
    align_items='stretch',
    width='50%'
))

In [6]:
form

Box(children=(Box(children=(Label(value='Первая колонка'), Select(options=('Unnamed: 0', 'AveragePrice', 'Tota…

In [7]:
def MakeNClusters(n):
    left = form.children[0].children[1].value
    right = form.children[1].children[1].value
    X = ddf[[left, right]].values
    X = StandardScaler().fit_transform(df[[left, right]].values)
    kmeans = KMeans(n_clusters=n, init='k-means++', max_iter=300, n_init=10, random_state=0)
    pred_y = kmeans.fit(X)
    plt.scatter(X[:,0], X[:,1], c= kmeans.labels_.astype(float))
    plt.scatter(kmeans.cluster_centers_[:, 0], kmeans.cluster_centers_[:, 1], s=20, c='red')
    plt.title('Количество кластеров: %d' % n)
    plt.show()

In [8]:
interactive_plot = interactive(MakeNClusters, n=(2, 10))
display(interactive_plot)

interactive(children=(IntSlider(value=6, description='n', max=10, min=2), Output()), _dom_classes=('widget-int…

In [9]:
def MakeDBSCAN(eps):
    left = form.children[0].children[1].value
    right = form.children[1].children[1].value
    X = StandardScaler().fit_transform(df[[left, right]].values)
    db = DBSCAN(eps=eps, min_samples=10).fit(X)

    core_samples_mask = np.zeros_like(db.labels_, dtype=bool)
    core_samples_mask[db.core_sample_indices_] = True
    labels = db.labels_

    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
    n_noise_ = list(labels).count(-1)

    unique_labels = set(labels)
    colors = [plt.cm.Spectral(each)
              for each in np.linspace(0, 1, len(unique_labels))]
    for k, col in zip(unique_labels, colors):
        if k == -1:
            # Black used for noise.
            col = [0, 0, 0, 1]

        class_member_mask = (labels == k)

        xy = X[class_member_mask & core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=14)

        xy = X[class_member_mask & ~core_samples_mask]
        plt.plot(xy[:, 0], xy[:, 1], 'o', markerfacecolor=tuple(col),
                 markeredgecolor='k', markersize=6)

    plt.title('Количество кластеров: %d' % n_clusters_)
    plt.show()

In [10]:
interactive_plot = interactive(MakeDBSCAN, eps=(0.1, 1.0))
display(interactive_plot)

interactive(children=(FloatSlider(value=0.55, description='eps', max=1.0, min=0.1), Output()), _dom_classes=('…

In [11]:
form

Box(children=(Box(children=(Label(value='Первая колонка'), Select(index=1, options=('Unnamed: 0', 'AveragePric…

In [12]:
def MeanShift(bd):
    left = form.children[0].children[1].value
    right = form.children[1].children[1].value
    X = StandardScaler().fit_transform(df[[left, right]].values)
    ms = cluster.MeanShift(bandwidth=bd, bin_seeding=True)
    ms.fit(X)

    cluster_centers_indices = ms.cluster_centers_
    n_clusters_ = len(cluster_centers_indices)
    labels = ms.labels_

    cluster_centers = ms.cluster_centers_

    for k in range(n_clusters_):
        my_members = labels == k
        cluster_center = cluster_centers[k]
        plt.plot(X[my_members, 0], X[my_members, 1])
        plt.plot(cluster_center[0], cluster_center[1], 'o',
                 markeredgecolor='k', markersize=5)
    plt.title('Количество кластеров: %d' % n_clusters_)
    plt.show()

In [13]:
interactive_plot = interactive(MeanShift, bd=(0.0, 5))
display(interactive_plot)

interactive(children=(FloatSlider(value=2.5, description='bd', max=5.0), Output()), _dom_classes=('widget-inte…